测试HBase的运行
需要提前启动HBase的thrift服务：
${HBASE_HOME}/bin/hbase-daemon.sh start thrift -threadpool

In [68]:
# 执行这个之前，要确保hbase的thrift的服务是打开的
import happybase
conn = happybase.Connection('192.168.17.150', autoconnect=True)
conn.open()
conn.tables()

[b'spark', b'test', b'zzti']

In [27]:
# 创建表
families = {
    'd': dict(max_versions=1, block_cache_enabled=False)
}
conn.create_table("test", families)

In [69]:
table = conn.table("test")
table.families()

{b'd': {'name': b'd:',
  'max_versions': 1,
  'compression': b'NONE',
  'in_memory': False,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'bloom_filter_nb_hashes': 0,
  'block_cache_enabled': False,
  'time_to_live': 2147483647}}

In [35]:
table.put(row="1",
          data={"d:a":"hello"},
          timestamp=1,
          wal=False)

In [36]:
# 写入数据
import pyhdfs
fs = pyhdfs.HdfsClient(user_name='zzti')
dir_input = "/test/input/"
sending_arr = []
for file in fs.listdir(dir_input):
    for line in fs.open(dir_input+file):
        sending_arr += str(line, encoding="utf-8").split(" ")
for i in range(len(sending_arr)):
    table.put(row=str(i),
              data={"d:word":sending_arr[i]},
              timestamp=1,
              wal=False)


In [ ]:
# 查询数据
print(table.row(row="1"))
print(table.rows(rows=["1","2","3"]))

In [66]:
dd = dict({"a":"b"})
print([ x for x in [1,2,3]])
print([ (x,dd[x]) for x in dd ])

[1, 2, 3]
[('a', 'b')]


In [72]:
# 查询数据
for row,val in table.scan(row_start="10", row_stop="20"):
    val_str = ",".join([ (c.decode()+": "+val[c].decode()) for c in val ])
    print(str(row,encoding="utf8"),val_str)

10 d:word: will
100 d:word: the
1000 d:word: Alabama,
1001 d:word: go
1002 d:word: back
1003 d:word: to
1004 d:word: South
1005 d:word: Carolina,
1006 d:word: go
1007 d:word: back
1008 d:word: to
1009 d:word: Georgia,
101 d:word: life
1010 d:word: go
1011 d:word: back
1012 d:word: to
1013 d:word: Louisiana,
1014 d:word: go
1015 d:word: back
1016 d:word: to
1017 d:word: the
1018 d:word: slums
1019 d:word: and
102 d:word: of
1020 d:word: ghettos
1021 d:word: of
1022 d:word: our
1023 d:word: northern
1024 d:word: cities,
1025 d:word: knowing
1026 d:word: that
1027 d:word: somehow
1028 d:word: this
1029 d:word: situation
103 d:word: the
1030 d:word: can
1031 d:word: and
1032 d:word: will
1033 d:word: be
1034 d:word: changed.

1035 d:word: Let
1036 d:word: us
1037 d:word: not
1038 d:word: wallow
1039 d:word: in
104 d:word: Negro
1040 d:word: the
1041 d:word: valley
1042 d:word: of
1043 d:word: despair,
1044 d:word: I
1045 d:word: say
1046 d:word: to
1047 d:word: you
1048 d:word: today,
1049

In [73]:
# 删除表
print(conn.tables())
conn.disable_table("test")
conn.delete_table("test")
print(conn.tables())

[b'spark', b'zzti']